In [20]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True,nb_workers=24)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [21]:
df = pd.read_csv('data/data.csv')

In [22]:
df = df.drop_duplicates('text').reset_index(drop=True)

In [26]:
from difflib import get_close_matches
patterns = df.text.to_list()
def closeMatches(word):
    global patterns
    return get_close_matches(word,patterns,n=2,cutoff=0.85)

In [27]:
df['closest'] = df.text.parallel_apply(lambda x: closeMatches(x))

In [28]:
df[['match1','match2']] = pd.DataFrame(df['closest'].to_list(), columns=['team1','team2'])

In [29]:
df.loc[df.text==df.match1, 'match1'] = np.nan

In [30]:
df.drop('match1',axis=1,inplace=True)

In [31]:
df.isnull().sum()

id                  0
text                0
is_offensive        0
target              0
text_len            0
target_label        0
public_fold         0
private_fold        0
closest             0
match2          11462
dtype: int64

In [32]:
df['match2_len'] = df['match2'].str.len().fillna(0).astype(int)

In [33]:
df

,id,text,is_offensive,target,text_len,target_label,public_fold,private_fold,closest,match2,match2_len
0,81c11060-a240-4d54-841b-9e2916039e85,çürük dişli,1,INSULT,11,4,1,0,[çürük dişli],None,0
1,be80ebbf-b322-4c3b-afa1-94932ea80731,bu adamın islama ve müslümanlara verdiği zarar...,1,RACIST,67,3,3,0,[bu adamın islama ve müslümanlara verdiği zara...,None,0
2,f99e2513-83ed-4076-ac72-b9e2cff3f049,erkekler zora gelmez,1,SEXIST,20,2,2,0,[erkekler zora gelmez],None,0
3,83ed2b2e-b815-4f36-9fc4-80a9050cf2d0,utanmazın götüne kazık sokmuşlar bu tıkırtı ne...,1,PROFANITY,65,1,4,0,[utanmazın götüne kazık sokmuşlar bu tıkırtı n...,None,0
4,d93e05f7-bfdd-4cdb-99d8-3048761b30ff,otomasyon< sistemlerine= doğrudan bağlanabilir,0,OTHER,46,0,4,0,[otomasyon< sistemlerine= doğrudan bağlanabili...,otomasyon sistemlerine doğrudan bağlanabilir,44
...,...,...,...,...,...,...,...,...,...,...,...
12400,71eedfa1-8fa6-425c-b982-258c3b29c003,uyuma taklidi yapan tehlikeli bir hayvanın göz...,0,OTHER,63,0,4,1,[uyuma taklidi yapan tehlikeli bir hayvanın gö...,None,0
12401,b38eed16-6501-4563-8b33-ff2e634bb8e5,yolda at kavga eden üç oğlan çocuğu görür,0,OTHER,41,0,4,1,[yolda at kavga eden üç oğlan çocuğu görür],None,0
12402,c8a051a8-94ef-4b64-a48e-54d0fa4f8323,sizin köpeklerinizin burnu bile daha iyi koku ...,0,OTHER,66,0,4,0,[sizin köpeklerinizin burnu bile daha iyi koku...,None,0
12403,513a7e6d-4207-4a16-9b47-972f26e23cfe,hayalleri gerçek etmek için birisinin delilik ...,0,OTHER,90,0,1,4,[hayalleri gerçek etmek için birisinin delilik...,None,0


In [34]:
df_sub = df.dropna(subset=['match2'])

In [35]:
df_sub

,id,text,is_offensive,target,text_len,target_label,public_fold,private_fold,closest,match2,match2_len
4,d93e05f7-bfdd-4cdb-99d8-3048761b30ff,otomasyon< sistemlerine= doğrudan bağlanabilir,0,OTHER,46,0,4,0,[otomasyon< sistemlerine= doğrudan bağlanabili...,otomasyon sistemlerine doğrudan bağlanabilir,44
22,c99b56c2-f683-48ac-90d4-f540f64c9328,köpek gibi ne havlayıp duruyorsun,1,INSULT,33,4,2,4,"[köpek gibi ne havlayıp duruyorsun, köpek gibi...",köpek gibi havlayıpcduruyorsun,30
37,4ecb25c0-b2f7-4b82-bfd9-832549e4fd74,agzina salincak kurar sallana sallana sicarim,1,PROFANITY,45,1,4,2,[agzina salincak kurar sallana sallana sicarim...,ağzına salıncak kurarsallana sallana sıçarım,44
103,0f512230-451f-4ccb-ba34-81570280e915,ananı sikerim baban olurum,1,PROFANITY,26,1,4,1,"[ananı sikerim baban olurum, ananı siker baban...",ananı siker baban olurum,24
106,7d60c943-ada2-4f6f-9416-a087f8848403,araplar haindir.,1,RACIST,16,3,3,2,"[araplar haindir., araplar haindir]",araplar haindir,15
...,...,...,...,...,...,...,...,...,...,...,...
12206,f26d177e-7c99-46ca-8469-f639519c675a,kadın kocasının ayaklarını yıkamalı,1,SEXIST,35,2,4,1,"[kadın kocasının ayaklarını yıkamalı, kadın ko...",kadın kocasının ayağını yıkamalıdır,35
12249,0a391287-39d7-4160-8ee5-343013fab4d2,orospunun çıkartıp kerhane köşelerine attığı k...,1,PROFANITY,55,1,2,2,[orospunun çıkartıp kerhane köşelerine attığı ...,orospunun çıkartıp kerhane köşelerine attigi k...,55
12254,83d7088b-85ab-4570-8419-aee7d5e6479b,kürt’ün kocayanına keçi güttürürler,1,RACIST,35,3,2,4,"[kürt’ün kocayanına keçi güttürürler, kürdün k...",kürdün kocayanına keçi güttürürler,34
12259,17f38489-65e0-47f3-802c-ea95e953b7f3,kadınsan kadınlığını bileceksin,1,SEXIST,31,2,1,3,"[kadınsan kadınlığını bileceksin, kadın kadınl...",kadın kadınlığını bilecek,25


In [36]:
df_sub['len_diff'] = abs(df_sub['text_len'] - df_sub['match2_len'])

/tmp/ipykernel_7541/1859431624.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['len_diff'] = abs(df_sub['text_len'] - df_sub['match2_len'])


In [37]:
df_sub = df_sub[df_sub['len_diff']<4].merge(df[['id','text','target']].rename(columns={'text':'match2'}),how='left',on='match2')

In [38]:
df_sub = df_sub[['id_x','text','target_x','text_len','closest','id_y','match2','match2_len','len_diff','target_y']]

In [44]:
df_sub.head()

,id_x,text,target_x,text_len,closest,id_y,match2,match2_len,len_diff,target_y
0,d93e05f7-bfdd-4cdb-99d8-3048761b30ff,otomasyon< sistemlerine= doğrudan bağlanabilir,OTHER,46,[otomasyon< sistemlerine= doğrudan bağlanabili...,d9fd2463-2ab4-443f-9e2f-ddcd8be84740,otomasyon sistemlerine doğrudan bağlanabilir,44,2,OTHER
1,c99b56c2-f683-48ac-90d4-f540f64c9328,köpek gibi ne havlayıp duruyorsun,INSULT,33,"[köpek gibi ne havlayıp duruyorsun, köpek gibi...",52093506-3a22-4614-aa40-e7ae5163fc94,köpek gibi havlayıpcduruyorsun,30,3,INSULT
2,4ecb25c0-b2f7-4b82-bfd9-832549e4fd74,agzina salincak kurar sallana sallana sicarim,PROFANITY,45,[agzina salincak kurar sallana sallana sicarim...,d442f69d-1fb5-4963-b31a-2b5cc35e50e2,ağzına salıncak kurarsallana sallana sıçarım,44,1,PROFANITY
3,0f512230-451f-4ccb-ba34-81570280e915,ananı sikerim baban olurum,PROFANITY,26,"[ananı sikerim baban olurum, ananı siker baban...",38402b8e-e23c-4871-ae97-14eeb15ea152,ananı siker baban olurum,24,2,PROFANITY
4,7d60c943-ada2-4f6f-9416-a087f8848403,araplar haindir.,RACIST,16,"[araplar haindir., araplar haindir]",35c47bfd-c5ab-43f8-a922-e67e0ccb446d,araplar haindir,15,1,RACIST


In [40]:
(df_sub.target_x == df_sub.target_y).value_counts()

True     640
False     29
dtype: int64

In [39]:
df_sub.shape

(669, 10)

In [42]:
df_sub.to_csv('data/duplicates.csv')